# V2 Model with Class Balancing via undersampling

For this, I used the exact same model build and process as in version 2, however I used a training set with completely balanced classes by removing any extra samples

Note that this method of class balancing does not guarantee an even spread of the subclassess, which could mess with the final results.

In [1]:
# imports
import numpy as np
import tensorflow as tf
import os

# load data
Xtrain = np.load("../data/train/Xtrainnew.npy")
Ytrain = np.load("../data/train/Ytrainnew.npy")
Xtest = np.load("../data/test/Xtestnew.npy")
Ytest = np.load("../data/test/Ytestnew.npy")

# undersample for even class balance
Ytrain_use = Ytrain[:1124]
Xtrain_use = Xtrain[:1124]
Ytest_use = Ytest[:282]
Xtest_use = Xtest[:282]

2023-03-31 10:40:01.445130: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# setup CNN using Inception-v3 (using google JAMA paper for this choice)
base_model = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=Xtrain.shape[1:]
)

# freeze inceptionv3 model
for layer in base_model.layers:
    layer.trainable = False

# add additional layers to iterate on
inputs = tf.keras.Input(shape=Xtrain.shape[1:])
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x) #sumarizing feature existence rather than location
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)


from keras.callbacks import EarlyStopping, ReduceLROnPlateau
# add some training callbacks
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3, verbose=1, mode='auto')
# Reducing the Learning Rate if result is not improving. 
reduce_lr = ReduceLROnPlateau(monitor='val_loss', min_delta=0.0004, patience=2, factor=0.1, min_lr=1e-6, mode='auto',
                              verbose=1)

2023-03-31 10:40:07.856307: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# train
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.AUC()]) 
#AUC: represents the probability that a random positive (green) example is positioned to the right of a random negative (red) example
model.fit(x=Xtrain_use, y=Ytrain_use, epochs=20, validation_data=(Xtest_use, Ytest_use), callbacks = [early_stop, reduce_lr])

Epoch 1/20
36/36 [==============================] - 20s 464ms/step - loss: 3.5869 - auc_1: 0.5667 - val_loss: 2.1707 - val_auc_1: 0.5446 - lr: 0.0010
Epoch 2/20
36/36 [==============================] - 17s 465ms/step - loss: 1.4605 - auc_1: 0.6529 - val_loss: 2.0496 - val_auc_1: 0.5289 - lr: 0.0010
Epoch 3/20
36/36 [==============================] - 17s 474ms/step - loss: 1.0970 - auc_1: 0.6749 - val_loss: 2.0814 - val_auc_1: 0.5369 - lr: 0.0010
Epoch 4/20
36/36 [==============================] - ETA: 0s - loss: 1.0666 - auc_1: 0.6896
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
36/36 [==============================] - 16s 453ms/step - loss: 1.0666 - auc_1: 0.6896 - val_loss: 2.7954 - val_auc_1: 0.5226 - lr: 0.0010
Epoch 5/20
36/36 [==============================] - 16s 454ms/step - loss: 1.0054 - auc_1: 0.6887 - val_loss: 1.9704 - val_auc_1: 0.5514 - lr: 1.0000e-04
Epoch 6/20
36/36 [==============================] - 16s 458ms/step - loss: 0.7820 - auc_1

In [7]:
# save model
model.save("../data/saved_model_balanced")

INFO:tensorflow:Assets written to: ../data/saved_model_balanced/assets


INFO:tensorflow:Assets written to: ../data/saved_model_balanced/assets


In [8]:
# raw accuracy
y_pred = model.predict(Xtest, batch_size=1)
accuracy = 0
for i in range(len(y_pred)):
    if np.round(y_pred[i]) == np.round(Ytest[i]):
        accuracy += 1
print(f"accuracy: {1-(accuracy/len(y_pred))}")

# get sensitivity and specificity, 0 is pos in orig data
sens = 0
sens_tot = 0
spec = 0
spec_tot = 0
for i in range(len(y_pred)):
    pred = np.round(y_pred[i])
    tru = np.round(Ytest[i])
    # sensitivity: number of correct positives out of all positives
    if tru == 0:
        sens_tot += 1
        if tru != pred:
            sens += 1
    # specificity: number of correct negatives out of all negatives
    elif tru == 1:
        spec_tot += 1
        if tru != pred:
            spec += 1
#display both
print(f"Sensitivity: {sens/sens_tot}")
print(f"Specificity: {spec/spec_tot}")

887/887 [==============================] - 22s 23ms/step
accuracy: 0.9143179255918827
Sensitivity: 0.9290780141843972
Specificity: 0.9115281501340483


## Next Steps
Knowing I want to use transfer learning going forward and still wanting to improve upon my ROC-AUC score, going forward I would play with the type of image preprocessing I would do and the layers that I added on top of InceptionV3 to see how that would improve my model. Some ideas include additional noise reduction on my images, as the final images still contain a lot of gray fuzz. Additionally, I would want to play around with using different added layer types in addition to the Global Pooling, consulting more references online for what has worked in solving similar problems in the past.

Additionally, overfitting and class imbalance problems are still a potential issue here. While I do already implement early stopping for overfitting, I would want to try playing with larger learning rates to see if I can do any better. For the class imbalance issue, I would also want to try training a model on undersampled data (taking a proportional number of images from each subclass for a representative sample) with an even class distribution to judge its performance. 